# 📝 Template Generation Tutorial

This tutorial focuses on **generating BuildingMOTIF templates** from semantic objects and using them to create RDF models.

## What You'll Learn

1. **Template structure** and how it's generated from classes
2. **Exporting templates** to YAML files
3. **Template dependencies** and parameter handling
4. **BMotifSession workflow** for model building
5. **Advanced template patterns** and customization
6. **Integration with BuildingMOTIF** ecosystem

Templates are the bridge between your Python semantic objects and RDF models! 🌉

## Setup and Imports

In [ ]:
from semantic_objects.s223 import *
from semantic_objects.core import export_templates, get_related_classes
from semantic_objects.build_model import BMotifSession
from pprint import pprint
import yaml
import os
from pathlib import Path

print("✅ Imports successful!")

## 1. 🔍 Understanding Template Structure

Let's start by examining how templates are generated from semantic object classes:

In [ ]:
# Look at the template parameters for Space
print("Space template parameters:")
space_params = Space._get_template_parameters()
pprint(space_params)

print("\n" + "="*50)
print("Window template parameters:")
window_params = Window._get_template_parameters()
pprint(window_params)

In [ ]:
# Generate the Turtle body for Space template
print("Space template body (Turtle/RDF):")
space_body = Space.generate_turtle_body()
print(space_body)

In [ ]:
# Look at template dependencies
print("Space template dependencies:")
space_deps = Space.get_dependencies()
pprint(space_deps)

print("\nWindow template dependencies:")
window_deps = Window.get_dependencies()
pprint(window_deps)

## 2. 📄 Generating YAML Templates

Now let's generate complete YAML templates that BuildingMOTIF can use:

In [ ]:
# Generate YAML template for Space
print("Space YAML template:")
space_yaml = Space.generate_yaml_template()
print(space_yaml)

print("\n" + "="*60)
print("Window YAML template:")
window_yaml = Window.generate_yaml_template()
print(window_yaml)

## 3. 🗂️ Exporting Template Collections

The `export_templates` function creates organized template files:

In [ ]:
# Export templates for Space (includes all related classes)
export_templates(Space, 'space_templates')

print("✅ Exported Space templates")
print("Generated files:")
for file in os.listdir('space_templates'):
    print(f"  - {file}")
    
# Show file sizes
for file in os.listdir('space_templates'):
    size = os.path.getsize(f'space_templates/{file}')
    print(f"    {file}: {size} bytes")

In [ ]:
# Look at the contents of each template file
template_files = ['entities.yml', 'relations.yml', 'values.yml']

for file in template_files:
    file_path = f'space_templates/{file}'
    if os.path.exists(file_path):
        print(f"\n=== {file} ===")
        with open(file_path, 'r') as f:
            content = f.read()
            # Show first 500 characters
            print(content[:500] + "..." if len(content) > 500 else content)

## 4. 🔗 Understanding Related Classes

When you export templates, the system finds all related classes automatically:

In [ ]:
# Get all related classes for Space
predicates, entities, values = get_related_classes(Space)

print(f"Found {len(predicates)} predicates, {len(entities)} entities, {len(values)} values")

print("\nPredicates (Relations):")
for pred in predicates:
    print(f"  - {pred.__name__}")

print("\nEntities:")
for entity in entities:
    print(f"  - {entity.__name__}")

print("\nValues (Properties):")
for value in values:
    print(f"  - {value.__name__}")

In [ ]:
# Export templates for Window (more complex)
export_templates(Window, 'window_templates')

# Compare the related classes
window_predicates, window_entities, window_values = get_related_classes(Window)

print(f"Window related classes:")
print(f"  Predicates: {len(window_predicates)}")
print(f"  Entities: {len(window_entities)}")
print(f"  Values: {len(window_values)}")

print("\nWindow-specific values:")
for value in window_values:
    print(f"  - {value.__name__}")

## 5. 🏗️ BMotifSession Workflow

Now let's use the generated templates with BMotifSession to build models:

In [ ]:
# Create a BMotifSession
session = BMotifSession(ns='template_demo')

# Load templates for Space
session.load_class_templates(Space)

print("✅ BMotifSession created")
print(f"Loaded templates: {list(session.templates.keys())}")

# Examine a template
space_template = session.templates['Space']
print(f"\nSpace template parameters: {space_template.all_parameters}")

In [ ]:
# Look at template with dependencies inlined
inlined_template = space_template.inline_dependencies()
print("Space template with inlined dependencies:")
print(f"Parameters: {inlined_template.all_parameters}")
print(f"Body preview: {str(inlined_template.body)[:200]}...")

In [ ]:
# Create and evaluate a space
office_space = Space(area=180.0)
office_space._name = "Office_301"

print("Space field values before evaluation:")
field_values = office_space.get_field_values(recursive=True)
pprint(field_values)

# Evaluate the space
session.evaluate(office_space)
print("\n✅ Space evaluated and added to model")

In [ ]:
# Examine the generated RDF
print("Generated RDF model:")
rdf_output = session.graph.serialize(format='turtle')
print(rdf_output)

## 6. 🔄 Multiple Object Workflow

Let's create a more complex model with multiple objects:

In [ ]:
# Create a new session for multiple objects
building_session = BMotifSession(ns='building_demo')

# Load templates for both Space and Window
building_session.load_class_templates(Space)
building_session.load_class_templates(Window)

print(f"Loaded templates: {list(building_session.templates.keys())}")

In [ ]:
# Create multiple spaces
spaces = [
    Space(area=120.0),  # Office 1
    Space(area=200.0),  # Conference room
    Space(area=80.0),   # Storage
]

space_names = ["Office_A", "Conference_B", "Storage_C"]
for space, name in zip(spaces, space_names):
    space._name = name

# Create windows for the spaces
windows = [
    Window(area=15.0, azimuth=90.0, tilt=90.0),   # East
    Window(area=25.0, azimuth=180.0, tilt=90.0),  # South
    Window(area=8.0, azimuth=270.0, tilt=90.0),   # West
]

window_names = ["Win_East_A", "Win_South_B", "Win_West_C"]
for window, name in zip(windows, window_names):
    window._name = name

print(f"Created {len(spaces)} spaces and {len(windows)} windows")

In [ ]:
# Evaluate all objects
print("Evaluating spaces...")
for space in spaces:
    building_session.evaluate(space)
    print(f"  ✅ {space._name}: {space.area.value} ft²")

print("\nEvaluating windows...")
for window in windows:
    building_session.evaluate(window)
    direction = {90.0: "East", 180.0: "South", 270.0: "West"}[window.azimuth.value]
    print(f"  ✅ {window._name}: {window.area.value} ft², {direction}")

print(f"\nTotal triples in model: {len(building_session.graph)}")

## 7. 📊 Template Analysis and Debugging

Let's analyze the templates and understand how they work:

In [ ]:
# Analyze template structure
print("Template Analysis:")
for template_name, template in building_session.templates.items():
    print(f"\n{template_name}:")
    print(f"  Parameters: {template.all_parameters}")
    print(f"  Dependencies: {[dep.template.name for dep in template.dependencies]}")
    
    # Show body preview
    body_str = str(template.body)
    lines = body_str.split('\n')
    print(f"  Body lines: {len(lines)}")
    print(f"  Body preview: {lines[0] if lines else 'Empty'}")

In [ ]:
# Look at parameter flattening for complex objects
complex_space = Space(area=150.0)
complex_space._name = "Complex_Space"

print("Field values (nested):")
nested_values = complex_space.get_field_values(recursive=True)
pprint(nested_values)

# Show how BMotifSession flattens these for template evaluation
print("\nHow BMotifSession would flatten this:")
# This is what happens inside session.evaluate()
from collections.abc import Mapping
from rdflib import URIRef, Literal

def flatten_dict(d, parent_key="", sep="-", out=None):
    if out is None:
        out = {}
    
    for k, v in d.items():
        if k == "_name":
            if parent_key == "":
                out["name"] = building_session.building_ns[v]
            else:
                out[parent_key] = building_session.building_ns[v]
            continue
            
        if not isinstance(v, URIRef) and not isinstance(v, dict):
            v = Literal(v)
            
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        
        if isinstance(v, Mapping):
            flatten_dict(v, new_key, sep=sep, out=out)
        else:
            out[new_key] = v
    
    return out

flattened = flatten_dict(nested_values)
pprint(flattened)

## 8. 🎨 Custom Template Patterns

Let's explore some advanced template patterns:

In [ ]:
# Create a custom entity with multiple properties
from semantic_objects.s223.properties import Temperature
from semantic_objects.core import semantic_object, required_field

@semantic_object
class ConditionedSpace(Space):
    """A space with temperature control"""
    design_temperature: Temperature = required_field()

# Look at the generated template
print("ConditionedSpace template:")
conditioned_yaml = ConditionedSpace.generate_yaml_template()
print(conditioned_yaml)

In [ ]:
# Export templates for the custom class
export_templates(ConditionedSpace, 'custom_templates')

print("Custom template files:")
for file in os.listdir('custom_templates'):
    print(f"  - {file}")

# Check what's in the entities file
with open('custom_templates/entities.yml', 'r') as f:
    entities_content = f.read()
    print("\nEntities file content:")
    print(entities_content)

## 9. 🔧 Template Customization

You can customize how templates are generated:

In [ ]:
# Look at optional fields
print("Space optional fields:")
space_optional = Space.get_optional_fields()
print(space_optional)

print("\nWindow optional fields:")
window_optional = Window.get_optional_fields()
print(window_optional)

In [ ]:
# Create a class with optional fields
from semantic_objects.core import optional_field

@semantic_object
class FlexibleSpace(Space):
    """A space with optional properties"""
    volume: Optional[Volume] = optional_field(label="Space Volume")
    occupancy: Optional[int] = optional_field(label="Max Occupancy")

print("FlexibleSpace template:")
flexible_yaml = FlexibleSpace.generate_yaml_template()
print(flexible_yaml)

print("\nFlexibleSpace optional fields:")
print(FlexibleSpace.get_optional_fields())

## 10. 📁 Template File Management

Best practices for organizing and managing template files:

In [ ]:
# Create organized template structure
template_dir = Path('organized_templates')
template_dir.mkdir(exist_ok=True)

# Export different entity types to subdirectories
entities_to_export = {
    'spaces': [Space, ConditionedSpace],
    'windows': [Window],
    'properties': [Area, Temperature, Azimuth, Tilt]
}

for category, entity_list in entities_to_export.items():
    category_dir = template_dir / category
    category_dir.mkdir(exist_ok=True)
    
    for entity in entity_list:
        export_templates(entity, str(category_dir))
    
    print(f"✅ Exported {category} templates")

# Show the organized structure
print("\nOrganized template structure:")
for root, dirs, files in os.walk(template_dir):
    level = root.replace(str(template_dir), '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        print(f"{subindent}{file}")

In [ ]:
# Template versioning and metadata
def create_template_metadata(entity_class, version="1.0"):
    """Create metadata for a template"""
    metadata = {
        'template_name': entity_class.__name__,
        'version': version,
        'ontology': 's223',
        'description': entity_class.__doc__ or f"Template for {entity_class.__name__}",
        'parameters': list(entity_class._get_template_parameters().keys()),
        'dependencies': [dep['template'].__name__ for dep in entity_class.get_dependencies()],
        'optional_fields': entity_class.get_optional_fields()
    }
    return metadata

# Create metadata for our templates
space_metadata = create_template_metadata(Space)
window_metadata = create_template_metadata(Window)

print("Space template metadata:")
pprint(space_metadata)

print("\nWindow template metadata:")
pprint(window_metadata)

# Save metadata to files
with open('space_metadata.yml', 'w') as f:
    yaml.dump(space_metadata, f, default_flow_style=False)

with open('window_metadata.yml', 'w') as f:
    yaml.dump(window_metadata, f, default_flow_style=False)

print("\n✅ Saved template metadata files")

## 📊 Summary

Congratulations! You've mastered template generation with Semantic Objects:

✅ **Understood template structure** - parameters, body, and dependencies  
✅ **Generated YAML templates** from semantic object classes  
✅ **Exported organized template collections** to files  
✅ **Used BMotifSession** to build RDF models from templates  
✅ **Worked with complex objects** and multiple entities  
✅ **Analyzed template dependencies** and parameter flattening  
✅ **Created custom entities** with advanced patterns  
✅ **Organized template files** with best practices  
✅ **Added metadata and versioning** for template management  

### Key Template Concepts

1. **Automatic Generation**: Templates are generated automatically from class definitions
2. **Parameter Mapping**: Class fields become template parameters
3. **Dependency Resolution**: Related classes are included as dependencies
4. **RDF Generation**: Templates produce valid RDF when evaluated
5. **BuildingMOTIF Integration**: Templates work seamlessly with BuildingMOTIF

### Template Workflow

1. **Define** semantic object classes with fields
2. **Export** templates using `export_templates()`
3. **Load** templates in BMotifSession
4. **Create** object instances with data
5. **Evaluate** objects to generate RDF models

### Next Steps

- **Advanced Examples**: Complex relationships and custom patterns
- **Validation**: Use SHACL shapes with your templates
- **Integration**: Connect with BuildingMOTIF workflows
- **Custom Entities**: Create domain-specific semantic objects

Templates are the key to scaling semantic modeling - they let you define once and generate many! 🚀